# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,-6.77,77,100,2.56,GL,1713561329
1,1,port elizabeth,-33.9180,25.5701,18.51,95,20,1.03,ZA,1713561329
2,2,dudinka,69.4058,86.1778,-9.62,96,100,3.13,RU,1713561329
3,3,marsh harbour,26.5412,-77.0636,23.87,66,63,3.41,BS,1713561330
4,4,kill devil hills,36.0307,-75.6760,18.00,86,100,3.60,US,1713561330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

17


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,port-aux-francais,-49.3500,70.2167,3.83,62,58,9.02,TF,1713561331
26,26,whitehorse,60.7161,-135.0538,4.06,32,34,5.36,CA,1713561336
53,53,puerto natales,-51.7236,-72.4875,6.25,61,75,10.29,CL,1713561343
66,66,san carlos de bariloche,-41.1456,-71.3082,8.86,57,20,11.83,AR,1713561347
94,94,cold lake,54.4502,-110.2017,3.56,33,2,0.21,CA,1713561355


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,port-aux-francais,TF,-49.3500,70.2167,62,
26,whitehorse,CA,60.7161,-135.0538,32,
53,puerto natales,CL,-51.7236,-72.4875,61,
66,san carlos de bariloche,AR,-41.1456,-71.3082,57,
94,cold lake,CA,54.4502,-110.2017,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
san carlos de bariloche - nearest hotel: No hotel found
cold lake - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
mikuni - nearest hotel: No hotel found
dalandzadgad - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
altai - nearest hotel: No hotel found
fort st. john - nearest hotel: No hotel found
kharkhorin - nearest hotel: No hotel found
north battleford - nearest hotel: No hotel found
pamplona - nearest hotel: No hotel found
cromer - nearest hotel: No hotel found
fort mcmurray - nearest hotel: No hotel found
aurillac - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,port-aux-francais,TF,-49.3500,70.2167,62,No hotel found
26,whitehorse,CA,60.7161,-135.0538,32,No hotel found
53,puerto natales,CL,-51.7236,-72.4875,61,No hotel found
66,san carlos de bariloche,AR,-41.1456,-71.3082,57,No hotel found
94,cold lake,CA,54.4502,-110.2017,33,No hotel found
110,happy valley-goose bay,CA,53.3168,-60.3315,41,No hotel found
146,mikuni,JP,36.2171,136.1519,60,No hotel found
175,dalandzadgad,MN,43.5708,104.4250,54,No hotel found
221,punta arenas,CL,-53.1500,-70.9167,56,No hotel found
236,altai,MN,46.3722,96.2583,47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)